In [72]:
from selenium import webdriver
import time
import csv

#parser
class Scrapper():
    
    #initialize webdriver
    def __init__(self):
        path = 'D:\Programes\geckodriver_firefox\geckodriver.exe'        
        self.driver = webdriver.Firefox(executable_path=path)
    
    #processing page with current company
    def parse(self):
        self.driver.get('http://www.woos.ru')
        time.sleep(3)
        table = self.driver.find_element_by_xpath('//table/tbody/tr/td/table[@cellpadding="10"]/tbody/tr/td')
        disk = table.find_element_by_xpath('.//table/tbody/tr/td[2]/a')
        url = disk.get_attribute('href')
        name = disk.text
        
        print(url)
        print(name)


In [74]:
a = Scrapper()
a.parse()

http://www.woos.ru/7-rasa/index.html
7 раса


In [103]:
from selenium import webdriver
import time
import csv
import itertools

#parser
class Scrapper():
    
    #initialize webdriver
    def __init__(self):
        path = 'D:\Programes\geckodriver_firefox\geckodriver.exe'        
        self.driver = webdriver.Firefox(executable_path=path)
    
    def get_song(self, url_song, year):
        self.driver.get(url_song)
        time.sleep(1)
        try:
            table = self.driver.find_element_by_xpath('//table/tbody/tr/td/table[@cellpadding="10"]/tbody/tr/td')
            song_author = table.find_element_by_xpath('.//table[1]/tbody/tr/td[@class="h3"]').text
            song_album = table.find_element_by_xpath('.//table[2]/tbody/tr/td[@class="h2"]').text
            song_name = table.find_element_by_xpath('.//h1').text
            song_text = table.find_element_by_xpath('.//font').text
        except:
            print("problem with a song")
            return []
        
        res = [song_author, song_album, year, song_name, song_text]
        return res
    
    def get_songs(self, url_songs, year):
        self.driver.get(url_songs)
        time.sleep(1)
        pattern = "http://www.woos.ru/images/cass1.gif"
        
        table = self.driver.find_element_by_xpath('//table/tbody/tr/td/table[@cellpadding="10"]/tbody/tr/td')
        songs = table.find_elements_by_xpath('.//table[3]/tbody/tr')
        base_songs = dict()
        for song in songs:
            try:
                logo = song.find_element_by_xpath('.//td[2]/img').get_attribute('src')
                
                if logo == pattern:
                    continue
                song_info = song.find_element_by_xpath('.//td[3]/a')
                url = song_info.get_attribute('href')
                name = song_info.text
                base_songs[name] = url
            except:
                print("problem with the songs")
        res = []
        for song in base_songs:
            l = self.get_song(base_songs[song], year)
            res.append(l)

        return res
    
    def get_albums(self, url_albums):
        self.driver.get(url_albums)
        time.sleep(1)
        
        table = self.driver.find_element_by_xpath('//table/tbody/tr/td/table[@cellpadding="10"]/tbody/tr/td')
        albums = table.find_elements_by_xpath('.//table[3]/tbody/tr')
        base_albums = dict()
        for album in albums:
            try:
                album_info = album.find_element_by_xpath('.//td[3]/a')
                url = album.find_element_by_xpath('.//td[3]/a').get_attribute('href')
                name = album.find_element_by_xpath('.//td[3]').text
                base_albums[name] = url
            except:
                print("problem with an album")
        res = []
        for album in base_albums:
            l = self.get_songs(base_albums[album], int(album.rstrip()[-5:-1]))
            res.append(l)
        return list(itertools.chain.from_iterable(res))
            
    
    #processing page with current company
    def parse(self):
        self.driver.get('http://www.woos.ru')
        time.sleep(1)
        table = self.driver.find_element_by_xpath('//table/tbody/tr/td/table[@cellpadding="10"]/tbody/tr/td')
        groups = table.find_elements_by_xpath('./table/tbody/tr')
        base_groups = dict()
        for group in groups:
            try:
                groups_info = group.find_elements_by_xpath('.//td[2]/a')
                for group_info in groups_info:
                    url = group_info.get_attribute('href')
                    name = group_info.text
                    base_groups[name] = url
            except:
                print("problem with a group")
           
        print(len(base_groups))
        res = [['singer', 'album', 'album_year', 'song_name', 'song_text']]
        for group in base_groups:
            l = self.get_albums(base_groups[group])
            for line in l:
                res.append(line)
            
        
        #Запись данных в файл .csv    
        with open('output.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter='|', )
            for line in res:
                try:
                    writer.writerow(line)
                except:
                    pass
                
    def parse1(self):
        l = self.get_songs(r'http://www.woos.ru/smyslovye-gallyucinacii/02/index.html', 1997)
        print(l)

In [104]:
a = Scrapper()
a.parse()

130


In [107]:
list(itertools.chain.from_iterable([[1,2],[3,4, [5,6]]]))

[1, 2, 3, 4, [5, 6]]

In [105]:
[['singer1', 'album1', 'year1', 'song_name1', 'text_song1'], ['singer2', 'album2', 'year2', 'song_name2', 'text_song2'], ]

[['singer1', 'album1', 'year1', 'song_name1', 'text_song1'],
 ['singer2', 'album2', 'year2', 'song_name2', 'text_song2']]